## Simulação de uma LLM com memória da conversa (versão mais moderna do framework)

ref.: 
* https://python.langchain.com/docs/how_to/message_history/
* https://www.langchain.com/langgraph

**todo**:
- Tentar gerenciar a memória da conversa de forma automatizada

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import RemoveMessage

## Utilização do llama3.1 com interface via ollama

In [2]:
llm = ChatOllama(model="llama3.1:latest")
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

## Pergunta para iniciar a conversa

In [3]:
config = {"configurable": {"thread_id": "abc123"}}

query = "Quem foi Albert Einstein?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.

Seu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.


In [4]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106})]

## O modelo consegue se lembrar do que acabou de responder

In [5]:
query = "O que você disse?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Eu disse que Albert Einstein foi um dos físicos mais influentes de todos os tempos e que ele desenvolveu a Teoria da Relatividade, que mudou a forma como as pessoas pensam sobre espaço, tempo e gravidade. Além disso, mencionei que ele nasceu em 1879 e morreu em 1955.

Quer saber mais sobre sua vida ou suas contribuições para a física? Estou aqui para ajudar!


In [6]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que voc

In [7]:
query = "O que você disse? Ainda não consegui entender"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Vou tentar explicar de outra forma!

Albert Einstein foi um cientista famoso que viveu no século 20. Ele era muito inteligente e fez descobertas importantes sobre o universo.

Duas das coisas mais importantes que ele fez foram:

1. **Teoria da Relatividade**: Isso é como uma espécie de regra para entender como funciona o tempo e o espaço.
2. **Equação E=mc²**: Isso mostra que a energia (E) é igual à massa (m) multiplicada pela velocidade da luz (c) ao quadrado.

Em resumo, Einstein foi um cientista muito importante que fez coisas incríveis para entender o universo!


In [8]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que voc

In [9]:
query = "Nasceu onde?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Albert Einstein nasceu em **Munique**, na Alemanha, em 14 de março de 1879.


In [10]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que voc

In [11]:
query = "Quando?"
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

 Ele era filho de um comerciante judaico e cresceu em uma família modesta.

Mas, é importante notar que ele não passou toda a sua vida na Alemanha. Com o advento da Primeira Guerra Mundial e a ascensão do nazismo, Einstein se mudou para outros países, como a Suíça, os Estados Unidos e a Inglaterra, onde continuou a trabalhar em suas pesquisas científicas.


In [12]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que voc

## Testando o isolamento de sessões

In [13]:
# trocando a sessão
config = {"configurable": {"thread_id": "def234"}}

query = "O que você disse por último?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nada, esta é a primeira mensagem do nosso diálogo. Como podemos começar? Quer conversar sobre algo em particular ou você gostaria de me perguntar algo? Eu estou aqui para ajudar.


In [14]:
output["messages"]

[HumanMessage(content='O que você disse por último?', additional_kwargs={}, response_metadata={}, id='3b11085d-5935-4142-8742-28460bc53f77'),
 AIMessage(content='Nada, esta é a primeira mensagem do nosso diálogo. Como podemos começar? Quer conversar sobre algo em particular ou você gostaria de me perguntar algo? Eu estou aqui para ajudar.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:35.552126845Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3044092748, 'load_duration': 53233769, 'prompt_eval_count': 17, 'prompt_eval_duration': 143901000, 'eval_count': 47, 'eval_duration': 2845742000}, id='run-3048ec87-30ec-45a5-9401-e7ea1998f94d-0', usage_metadata={'input_tokens': 17, 'output_tokens': 47, 'total_tokens': 64})]

In [15]:
# retornando a sessão anterior
config = {"configurable": {"thread_id": "abc123"}}

query = "O que você disse por último?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Eu disse que Albert Einstein:

* Nasceu na Alemanha (em Munique)
* Se mudou para outros países como Suíça, EUA e Inglaterra devido à Primeira Guerra Mundial e ao nazismo.


In [16]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que voc

## Removendo as 2 primeiras interações (para não entupir de tokens)

In [17]:
messages = app.get_state(config).values["messages"]
messages

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='049fe12c-587a-4923-abfc-b6321947c782'),
 AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que voc

In [18]:
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})
messages = app.get_state(config).values["messages"]
messages

[AIMessage(content='Físico alemão nascido em 1879 e falecido em 1955. É considerado um dos mais influentes físicos da história, cujas teorias revolucionaram os conceitos de espaço, tempo e gravidade.\n\nSeu Teoria da Relatividade mudou o modo como pensamos sobre o universo, destacando-se por suas contribuições à Física Teórica.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:06.581946678Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5982654356, 'load_duration': 11165908, 'prompt_eval_count': 16, 'prompt_eval_duration': 387969000, 'eval_count': 90, 'eval_duration': 5532553000}, id='run-2d1fa02e-36a6-4960-97a7-26374c253d40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 90, 'total_tokens': 106}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='66f03933-b699-4871-9e70-6e18f5ccec76'),
 AIMessage(content='Eu disse que Albert 

In [19]:
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})
messages = app.get_state(config).values["messages"]
messages

[HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='66f03933-b699-4871-9e70-6e18f5ccec76'),
 AIMessage(content='Eu disse que Albert Einstein foi um dos físicos mais influentes de todos os tempos e que ele desenvolveu a Teoria da Relatividade, que mudou a forma como as pessoas pensam sobre espaço, tempo e gravidade. Além disso, mencionei que ele nasceu em 1879 e morreu em 1955.\n\nQuer saber mais sobre sua vida ou suas contribuições para a física? Estou aqui para ajudar!', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:13.172797258Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6574818512, 'load_duration': 63022471, 'prompt_eval_count': 120, 'prompt_eval_duration': 160489000, 'eval_count': 99, 'eval_duration': 6200748000}, id='run-35cd655d-c524-4c41-b27a-b9c365f48da9-0', usage_metadata={'input_tokens': 120, 'output_tokens': 99, 'total_t

## Retomando a conversa (para verificar se continua coerente)

In [20]:
query = "Qual foi a maior contribuição dele?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

A maior contribuição de Albert Einstein é provavelmente sua **Teoria da Relatividade**, que mudou a forma como as pessoas pensam sobre espaço, tempo e gravidade.

Essa teoria foi apresentada em 1905 e 1915, e ela revolucionou a física ao introduzir conceitos como:

* A igualdade entre massa e energia (E=mc²)
* O fato de que o tempo e o espaço são relativos e dependem da velocidade do observador
* A existência de buracos negros, que são regiões do espaço onde a gravidade é tão forte que nada, nem mesmo a luz, pode escapar.

Essa teoria foi um grande avanço para a física e mudou nossa compreensão do universo!


In [21]:
output["messages"]

[HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='66f03933-b699-4871-9e70-6e18f5ccec76'),
 AIMessage(content='Eu disse que Albert Einstein foi um dos físicos mais influentes de todos os tempos e que ele desenvolveu a Teoria da Relatividade, que mudou a forma como as pessoas pensam sobre espaço, tempo e gravidade. Além disso, mencionei que ele nasceu em 1879 e morreu em 1955.\n\nQuer saber mais sobre sua vida ou suas contribuições para a física? Estou aqui para ajudar!', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:17:13.172797258Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6574818512, 'load_duration': 63022471, 'prompt_eval_count': 120, 'prompt_eval_duration': 160489000, 'eval_count': 99, 'eval_duration': 6200748000}, id='run-35cd655d-c524-4c41-b27a-b9c365f48da9-0', usage_metadata={'input_tokens': 120, 'output_tokens': 99, 'total_t